In [1]:
import tensorflow as tf
import csv,cv2, numpy as np
import pandas as pd

In [12]:
# https://stanford.edu/~shervine/blog/keras-how-to-generate-data-on-the-fly
# https://github.com/shervinea/enzynet
import numpy as np
import keras
import matplotlib.pyplot as plt

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, labels, batch_size=32, dim=(32,32,32), shuffle=False, objectName = 'defaultName', **params):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        self.shuffle = shuffle
        self.on_epoch_end()
        self.indexes = list_IDs.index
        self.relativeLocation  = relativeLocation
        self.objectName = objectName
        
        print(self.indexes)


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        print(indexes)
        # Generate data
        X, y = self.__data_generation(indexes)
        return X, y

    def on_epoch_end(self,var = 'constructor'):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)
#         print(var, ' : reshuffling happening here')
        
    def __data_generation(self, indexes):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.empty((self.batch_size, *self.dim))
        y = np.empty((self.batch_size))
        
        i = 0
        for index in indexes:
            try:
                imageName = self.list_IDs[index]
                # index missing thus throw indexError
                steeringInput = self.labels[index]
                
                if(pd.isnull(imageName) or pd.isnull(steeringInput)):
                    print(imageName,steeringInput)
                    raise IndexError()
                
                try:
                    image_path = self.relativeLocation + 'IMG/' + imageName.split('/')[-1]
                    image = cv2.imread(image_path).reshape((1,160,320,3))
               
                    if image is None:
                        raise IndexError()
                    
                    image = image/255.0 - 0.5
                
                    X[i,] = image
                    y[i,] = steeringInput
                    i = i+1
                except IndexError:
                    print(self.objectName,' ,inner index ',index,' Either image or steering angle is missing')
                    pass
                except KeyError:
                    print(self.objectName,' ,inner key ',index,' Either image or steering angle is missing')
                    pass
        
        
        
            except IndexError:
                print(self.objectName,' ,outer index ',index,' Either image or steering angle is missing')
                pass
            except KeyError:
                print(self.objectName,' ,outer key ',index,' Either image or steering angle is missing')
                pass
        
        return X, y

In [16]:
import numpy as np
from sklearn.model_selection import train_test_split

# Parameters
params = {'dim': (160,320,3),
          'batch_size': 16,
          'shuffle': False,
          'relativeLocation':'../assignment_3/data_creation/'}

# Datasets
relativeLocation = params['relativeLocation']
df = pd.read_csv(relativeLocation + 'driving_log.csv')
X = df['center']
y = df['steering']

X_train, X_validation, y_train, y_validation = train_test_split(X, y, test_size=0.2)

training_generator = DataGenerator(X_train, y_train, objectName = 'trainingGenerator', **params)
validation_generator = DataGenerator(X_validation, y_validation, objectName = 'validationGenerator', **params)

# Some Debugging happening here
XX, yy = training_generator.__getitem__(0)
XX, yy = validation_generator.__getitem__(0)
# print(yy)
# for image in XX:
#     plt.figure(figsize=(10,10))
#     plt.imshow(image) # This is showig the image thinking it is a bgr image
#     print(image)
    


Int64Index([5605, 7887, 2958, 8109, 1079, 8541, 2017,  212, 2634,  353,
            ...
            6364, 2915, 1521, 9759,   58, 6674, 4504, 5909, 5679, 2478],
           dtype='int64', length=8024)
Int64Index([ 2229,  7638,   624,  5356,  5164,  9221,  6441,  4997,  5761,
             9765,
            ...
             6492,  5232, 10006,  6602,  7826,  5502,  3884,   854,  8659,
             9290],
           dtype='int64', length=2006)
Int64Index([5605, 7887, 2958, 8109, 1079, 8541, 2017,  212, 2634,  353,  484,
            1596, 7649, 2027,   33, 8333],
           dtype='int64')
Int64Index([2229, 7638,  624, 5356, 5164, 9221, 6441, 4997, 5761, 9765, 6127,
            5286, 8102, 1979, 5420, 1727],
           dtype='int64')


In [9]:
# '''
# Keral library imports
# '''
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Lambda, Cropping2D
from keras.layers import Conv2D, MaxPooling2D, Dropout

In [10]:
# ch, row, col = 3, 80, 320  # Trimmed image format
model = Sequential()
# model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3))) #Normalization layer
model.add(Cropping2D(cropping=((70,25),(0,0)),input_shape=(160,320,3)))
model.add(Conv2D(24, activation="relu", kernel_size=(5, 5), strides=(2, 2)))
# model.add(Conv2D(36, activation="relu", kernel_size=(5, 5), strides=(2, 2)))
# model.add(Conv2D(48, activation="relu", kernel_size=(5, 5), strides=(2, 2)))
# model.add(Conv2D(64, activation="relu", kernel_size=(3, 3)))
# model.add(Conv2D(64, activation="relu", kernel_size=(3, 3)))
model.add(Conv2D(8, activation="relu", kernel_size=(3, 3)))

model.add(Flatten())
# model.add(Dense(100))
# model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(1))

model.summary()

# # Network optimization choices set to mse and adam optimizer
model.compile(loss='mse',optimizer='adam')
          
# # Network training happening here
# # model..fit_generator(train_generator, samples_per_epoch=len(train_samples), validation_data=validation_generator,
# #                     nb_val_samples=len(validation_samples), epochs=3)
# # model.fit_generator(train_generator, steps_per_epoch= len(train_samples),
# # validation_data=validation_generator, validation_steps=len(validation_samples), epochs=5, verbose = 1)

# # model.save('model_3A.h5')


# # Design model
# model = Sequential()
# [...] # Architecture
# model.compile()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
cropping2d_2 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 31, 158, 24)       1824      
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 29, 156, 8)        1736      
_________________________________________________________________
flatten_2 (Flatten)          (None, 36192)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 10)                361930    
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 11        
Total params: 365,501
Trainable params: 365,501
Non-trainable params: 0
_________________________________________________________________


In [11]:
# Train model on dataset

print(X_train.index[0:20])
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=False,
                    shuffle = False,
                    steps_per_epoch = 10,
                    epochs = 2,
                    workers=1,
                    verbose=2)

model.save('model_generators_3A.h5')

Int64Index([6528, 9607, 9297, 5207, 3456, 4627, 7503, 6294, 9095,  406, 8298,
            3786, 7947, 6928, 5266, 6126, 7494, 2706, 5528, 2521],
           dtype='int64')
Epoch 1/2
Int64Index([9963, 2315, 4037, 6103, 9315, 4311, 7046, 9446, 8726, 3151, 7593,
            3905, 9878, 2042, 7269, 3494],
           dtype='int64')
Int64Index([6528, 9607, 9297, 5207, 3456, 4627, 7503, 6294, 9095,  406, 8298,
            3786, 7947, 6928, 5266, 6126],
           dtype='int64')
Int64Index([5875, 8669,  156, 7713, 5322, 9299, 8736, 8724, 3798, 4554, 7681,
            1179, 3806,  689, 1078,  170],
           dtype='int64')
Int64Index([6056, 9530,  980, 6643, 8444, 1639, 6743, 2321, 6168, 4579,  182,
            7665, 9447,  590, 8193, 7575],
           dtype='int64')
Int64Index([7494, 2706, 5528, 2521, 2160,  600, 3426, 3769, 1720, 9656, 7270,
            3427, 4399, 6003, 3320, 2430],
           dtype='int64')
Int64Index([ 577, 3036, 2795, 6147, 3131, 1020, 1139, 9792, 9871, 7633, 1622,
      

Int64Index([3176, 6237, 4158, 6887, 1760, 3339, 7897, 5418,   53, 2892, 3023,
            5647,  240, 1467, 1825, 8264],
           dtype='int64')
Int64Index([2702, 6489, 4702, 3823, 7709, 9416, 9358, 2588, 4456, 3634, 9145,
            8207, 9638, 7573, 9854, 1634],
           dtype='int64')
Int64Index([ 4343,  4138,  6015, 10020,  5048,  5790,  2571,  3242,  5159,
             8054,   417,  2734,  6035,  1197,  5340,   926],
           dtype='int64')
Int64Index([5442, 2011, 4471, 6321, 6676, 2503, 8084, 7775, 5909, 2634, 8706,
            7927, 3087, 8395, 6778, 3917],
           dtype='int64')
Int64Index([4965,  792, 6351, 5631, 7774, 9442, 4075, 2839, 3008, 7071, 1758,
            4470, 4776, 9527, 8545, 8228],
           dtype='int64')
Int64Index([1733, 2123, 7924, 2473, 8975, 3595, 5988, 4763, 9010, 4151, 3148,
            8450, 5259, 7218, 6832, 6703],
           dtype='int64')
Int64Index([8352, 5665, 1738, 9711, 3464, 6611, 7179, 4206, 7525, 5459, 1306,
            6306, 6130, 

Int64Index([2157, 6181, 1862, 6460, 3688,  985, 6283, 8854, 5467, 5280, 2657,
            8318, 2981, 1822, 5838, 3366],
           dtype='int64')
Int64Index([6323, 4472, 2476, 2255, 3797,  470, 7548, 3879,  483, 1440, 8577,
            3837, 1746,  305, 3072,  501],
           dtype='int64')
Int64Index([2152, 4613, 1724, 8861, 4873, 7109, 1138, 8898, 6956, 2537, 8383,
            6657, 9341, 1046, 7225,  296],
           dtype='int64')
Int64Index([5088, 4154, 3355, 7499, 2116, 5379, 8161,  954, 9579, 8948, 2730,
            5369, 2036, 5449, 3441, 7137],
           dtype='int64')
Int64Index([ 774, 2910, 7490, 9522, 4291, 7016, 1868, 9591, 1666, 3115, 1336,
             376,  327, 2434, 6988, 7510],
           dtype='int64')
Int64Index([7886, 9914, 4443, 5099, 3400, 8326, 4436, 4208, 3726, 5881, 2842,
            6902, 5883, 5126, 5127, 4205],
           dtype='int64')
Int64Index([9357, 8244, 2334, 8665, 6419, 9568, 3721, 9787, 4514, 5967, 5251,
            6279, 8668, 4531, 6526, 3020

validationGenerator  ,outer key  128  Either image or steering angle is missing
validationGenerator  ,outer key  129  Either image or steering angle is missing
validationGenerator  ,outer key  131  Either image or steering angle is missing
validationGenerator  ,outer key  132  Either image or steering angle is missing
validationGenerator  ,outer key  133  Either image or steering angle is missing
validationGenerator  ,outer key  135  Either image or steering angle is missing
validationGenerator  ,outer key  136  Either image or steering angle is missing
validationGenerator  ,outer key  137  Either image or steering angle is missing
validationGenerator  ,outer key  138  Either image or steering angle is missing
validationGenerator  ,outer key  139  Either image or steering angle is missing
validationGenerator  ,outer key  140  Either image or steering angle is missing
validationGenerator  ,outer key  141  Either image or steering angle is missing
validationGenerator  ,outer key  143  Ei

[320 321 322 323 324 325 326 327 328 329 330 331 332 333 334 335]Int64Index([ 344, 6087, 4380,  199, 2172, 7246, 5611, 3165, 9007, 8090, 2448,
            5033, 5842, 3372, 5149, 5907],
           dtype='int64')

validationGenerator  ,outer key  320  Either image or steering angle is missing
validationGenerator  ,outer key  321  Either image or steering angle is missing
validationGenerator  ,outer key  323  Either image or steering angle is missing
validationGenerator  ,outer key  324  Either image or steering angle is missing
validationGenerator  ,outer key  325  Either image or steering angle is missing
validationGenerator  ,outer key  328  Either image or steering angle is missing
validationGenerator  ,outer key  329  Either image or steering angle is missing
validationGenerator  ,outer key  330  Either image or steering angle is missing
validationGenerator  ,outer key  331  Either image or steering angle is missing
validationGenerator  ,outer key  332  Either image or steering angl

...

validationGenerator  ,outer key  1116  Either image or steering angle is missing
validationGenerator  ,outer key  1119  Either image or steering angle is missing
[1120 1121 1122 1123 1124 1125 1126 1127 1128 1129 1130 1131 1132 1133
 1134 1135]
validationGenerator  ,outer key  1120  Either image or steering angle is missing
validationGenerator  ,outer key  1122  Either image or steering angle is missing
validationGenerator  ,outer key  1123  Either image or steering angle is missing
validationGenerator  ,outer key  1124  Either image or steering angle is missing
validationGenerator  ,outer key  1125  Either image or steering angle is missing
validationGenerator  ,outer key  1126  Either image or steering angle is missing
validationGenerator  ,outer key  1127  Either image or steering angle is missing
validationGenerator  ,outer key  1128  Either image or steering angle is missing
validationGenerator  ,outer key  1129  Either image or steering angle is missing
validationGenerator  ,oute

validationGenerator  ,outer key  1324  Either image or steering angle is missing
validationGenerator  ,outer key  1327  Either image or steering angle is missing
[1328 1329 1330 1331 1332 1333 1334 1335 1336 1337 1338 1339 1340 1341
 1342 1343]
validationGenerator  ,outer key  1328  Either image or steering angle is missing
validationGenerator  ,outer key  1329  Either image or steering angle is missing
validationGenerator  ,outer key  1330  Either image or steering angle is missing
validationGenerator  ,outer key  1331  Either image or steering angle is missing
validationGenerator  ,outer key  1332  Either image or steering angle is missing
validationGenerator  ,outer key  1333  Either image or steering angle is missing
validationGenerator  ,outer key  1334  Either image or steering angle is missing
validationGenerator  ,outer key  1337  Either image or steering angle is missing
validationGenerator  ,outer key  1338  Either image or steering angle is missing
validationGenerator  ,oute

[1440 1441 1442 1443 1444 1445 1446 1447 1448 1449 1450 1451 1452 1453
 1454 1455]
validationGenerator  ,outer key  1441  Either image or steering angle is missing
validationGenerator  ,outer key  1442  Either image or steering angle is missing
validationGenerator  ,outer key  1443  Either image or steering angle is missing
validationGenerator  ,outer key  1444  Either image or steering angle is missing
validationGenerator  ,outer key  1445  Either image or steering angle is missing
validationGenerator  ,outer key  1446  Either image or steering angle is missing
validationGenerator  ,outer key  1447  Either image or steering angle is missing
validationGenerator  ,outer key  1448  Either image or steering angle is missing
validationGenerator  ,outer key  1449  Either image or steering angle is missing
validationGenerator  ,outer key  1450  Either image or steering angle is missing
validationGenerator  ,outer key  1451  Either image or steering angle is missing
validationGenerator  ,oute

validationGenerator  ,outer key  1669  Either image or steering angle is missing
validationGenerator  ,outer key  1670  Either image or steering angle is missing
validationGenerator  ,outer key  1671  Either image or steering angle is missing
validationGenerator  ,outer key  1672  Either image or steering angle is missing
validationGenerator  ,outer key  1673  Either image or steering angle is missing
validationGenerator  ,outer key  1674  Either image or steering angle is missing
validationGenerator  ,outer key  1675  Either image or steering angle is missing
validationGenerator  ,outer key  1676  Either image or steering angle is missing
validationGenerator  ,outer key  1677  Either image or steering angle is missing
validationGenerator  ,outer key  1679  Either image or steering angle is missing
[1680 1681 1682 1683 1684 1685 1686 1687 1688 1689 1690 1691 1692 1693
 1694 1695]
validationGenerator  ,outer key  1681  Either image or steering angle is missing
validationGenerator  ,oute

validationGenerator  ,outer key  1883  Either image or steering angle is missing
validationGenerator  ,outer key  1884  Either image or steering angle is missing
validationGenerator  ,outer key  1885  Either image or steering angle is missing
validationGenerator  ,outer key  1886  Either image or steering angle is missing
[1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901
 1902 1903]
validationGenerator  ,outer key  1888  Either image or steering angle is missing
validationGenerator  ,outer key  1889  Either image or steering angle is missing
validationGenerator  ,outer key  1890  Either image or steering angle is missing
validationGenerator  ,outer key  1891  Either image or steering angle is missing
validationGenerator  ,outer key  1892  Either image or steering angle is missing
validationGenerator  ,outer key  1893  Either image or steering angle is missing
validationGenerator  ,outer key  1894  Either image or steering angle is missing
validationGenerator  ,oute